# __ML in der Logistik__
# Exercise 03 - Decision trees

__Goal__: understand the algorithms decision tree, the concept of pruning, build a simple decision tree  algorithm in Python and use pruning.


__Data and application (Task 1.2)__: The dataset contains the Length, Duration and Route informations from AIS messages recorded on 2701 trips on 4 routes in North Europa (Bremerhaven > Hamburg, Felistowe > Rotterdam, Kiel > Gdynia, Rotterdam > Hamburg). We want to build a decision tree to predict the route using the other attributes.

__Data and application (Tasks 1.3 and 1.4, Part 2)__: The dataset contains the static information of 4503 trips recorded on 4 routes in North Europa (Rotterdam > Hamburg, Kiel > Gdynia, Bremerhaven > Hamburg, Felixstowe > Rotterdam). We want to build and prune a decision tree predicting if the trip was direct or made additional stops.



### Contents

* [1 Theory: understand the algorithm](#t1)
    * [1.1 The decision tree algorithm (20 min)](#t11)
    * [1.2 Drawing a tree (10 min)](#t12)
    * [1.3 Pruning (15 min)](#t13)
* [2 Practice: predict additional stops for a trip](#t2)
    * [2.1 The DecisionTreeClassifier algorithm (20 min)](#t21)
    * [2.2 Cost Complexity Pruning (20 min)](#t22)

## 1 Theory: understand the algorithm <a class="anchor" id="t1"></a>

### 1.1 The decision tree algorithm (20 min) <a class="anchor" id="t11"></a>

*Quickly explain the decision tree algorithm.*

: Supervised algorithm which can be used in classification and regression, this works by splitting the given data into subsets recursively based on the attribute values, and forming a tree.

*How does a decision tree handle continuous numeric attributes?*

: Sorts and choses based on threshold which gives an optimal tree (using Information Gain, Gini Index etc..)

*How can a tree handle a regression problem?*

: Using variance reduction or MSE instead of information gain or gini index, tree can be used for regression problem where leafs contains average of target values [piece wise constant approximation]

*How does the decision tree algorithm select an attribute to split on, for each node?*

: Information Gain (ID3 - Iterative Dichotomiser 3), Gini Index (CART - Classification and Regression Trees), Gain Ratio (C4.5), Variance Reduction (CART)

*How can a decision tree handle missing values for an attribute, in the learning and in the prediction phase?*

:
- Learning: can be ignored or can use weighted split based on how other instances with known values are distributed
- Prediction: can follow multiple paths proportionally or can use the most frequent path based on the learning data (remember weights)

*Is a decision tree robust to outliers? Why?*

: Somewhat robust, cause split is based on threshold the effect of an outlier is less severe.

*Explain the strategies of prepruning and postpruning.*

: 

- Prepruning: Stop growing if next split doesn't perform well, stops overfitting early but may stop soon (underfitting)
- Postpruning: Grow the full tree first and then cut the subtrees which doesn't perform well for accuracy (usually gives better generalization than prepruning)

*What is the result of each one of these two approaches for building a tree?*
+ *Select few big clusters, even if some instances are misclassified*
: Shallow nodes, simpler model and possibly underfits (prepruning)
+ *Select a lot of small clusters perfectly defining one class*
: More nodes, complex model and possibly overfits (no pruning)

### 1.2 Drawing a tree (10 min) <a class="anchor" id="t12"></a>

We are in possession of a dataset which contains the values for Length, Duration and Route for 2701 trips. We want to build a decision tree to classify the attribute Route. The dataset is represented on the following graph:

![text](03-building.PNG)

_According to the graph, which attribute should be selected for the first node of the tree? What would be the value of splitting for this node?_

: Duration and value would be 1000

_Draw a decision tree with a maximum depth of 4 which would classify this dataset (not perfect classification)._

![image.png](decision-tree-not-perfect-classification.png)

### 1.3 Pruning (15 min) <a class="anchor" id="t13"></a>

We are given the following dataset. It contains the static data for 4503 trips on the 4 usual routes, and a new attribute 'Direct' which states if the ship took a direct trip from the start port to the end port, or if it took additional stops in between. A value of 1 indicates that the trip was direct, while a value of 0 indicates one or more additional stops.

In [1]:
# Import and visualize the dataset

import pandas as pd
df = pd.read_csv('03-decision-tree_1.csv')
print(df.head())
print(df['Route'].unique())

   shiptype  Length  Breadth Draught        SOG  TotalDistance  Duration  \
0        71     269       32   10.64  11.830764     353.861375      1532   
1        71     323       43      12  13.203446     353.416043      1450   
2        71     376       48   13.23  14.779070     346.753233      1213   
3        71     300       49   12.83  13.668775     350.611945      1327   
4        70     145       25       ?  12.389986     369.674767      1535   

     Route  Direct  
0  ROT_HAM       1  
1  ROT_HAM       1  
2  ROT_HAM       1  
3  ROT_HAM       1  
4  ROT_HAM       1  
['ROT_HAM' 'KIE_GDY' 'FEL_ROT' 'BRE_HAM']


With this dataset, we want to build a decision tree that predicts the value of the attribute 'Direct'.

Following is a decision tree that has been built using the software Weka. We want to study the effect of pruning a decision tree.

Note 1: The tree being very complex, the right part of it has been cut for visualization. We focus here on the left part only (instances with TotalDistance <= 399)

Note 2: Each circle represents a node of the tree, each square is a leaf. 'Direct' stands for a classification with the value for the attribute 'Direct' = 1, Not direct is 'Direct' = 0. The values between parentheses stand for: (number of correctly classified instances / number of uncorrectly classified instances).

![text](03-pruning.png)

_What happens to a subtree which is pruned?_

__For each subtree, write down the repartition of the instances on the form: Direct-TP (true positive), Direct-FP (false positive), Not direct-TN (true negative), Not direct-FN (false negative). Write down your answers in the following code cell. (3 points)__

In [2]:
# Repartition of the instances

# Subtree1
t13_TP_1 = 4026
t13_FP_1 = 11
t13_TN_1 = 25
t13_FN_1 = 6

# Subtree2
t13_TP_2 = 14
t13_FP_2 = 0
t13_TN_2 = 21
t13_FN_2 = 0

# Subtree3
t13_TP_3 = 33
t13_FP_3 = 1
t13_TN_3 = 3
t13_FN_3 = 0

__Then, calculate the error rate that each subtree would give if it were transformed into a single leaf, as: error rate = (number of instances in the minimal class) / (total number of instances). Write down your answer in the following code cell. (3 points)__


here minority is class (target) with minimum cardinality in that particular subtree, (TP + FN - positive class, TN + FP - negative class)

In [3]:
# Error rate

# Subtree1
t13_error_1 = 36/4068

# Subtree2
t13_error_2 = 14/35

# Subtree3
t13_error_3 = 4/37

__From the error rates: which subtree should be pruned first? Write down your answer in the following code cell (the variable pruned should be an integer and equal to the number of the subtree you choose to prune). (2 points)__

In [4]:
# Pruned subtree

t13_pruned = 1

_What can we say about pruned trees?_

: since error rate is low we have pruned the tree, here it seems by visualizing most of the trips belong to direct rather than not direct and hence we can prune this and just classify as direct at the pruning node itself.

_What other methods can be used to prune a tree?_

: Subtree raising (deleting the inner node) and Cost complexity pruning

## 2 Practice: predict additional stops for a trip <a class="anchor" id="t2"></a>

### 2.1 The DecisionTreeClassifier algorithm (20 min) <a class="anchor" id="t21"></a>

For this task, we will use the dataset that we saw in tasks 1.3 and 1.4:

In [5]:
# Import the dataset

import pandas as pd
df = pd.read_csv('03-decision-tree_1.csv', na_values = '?')

For building a decision tree, we need all the attributes to be numerical. Therefore, we won't use the attribute 'Route' for this prediction. Also, the attribute 'Direct' should be categorical.

__Change the type of the attribute.__

In [7]:
# Change the type of the attribute
df.drop("Route", axis = 1)
df['Direct'] = df['Direct'].astype('category')

We want to build a simple decision tree model to predict the attribute 'Direct' from the static information of the trips. We will use the classifier ``DecisionTreeClassifier`` from the ``sklearn`` library. This classifier may use random values, so in this exercise we will specify the value for the parameter ``random_state = 1``.

The ``DecisionTreeClassifier`` can't process the attributes with missing values. If there are some in the dataset, identify them.

We want to predict the attribute 'Direct' using all the other attributes except 'Route' and any attribute with missing values.

For training and testing, we will use a 80-20% split.

__Write the code to apply the algorithm. Do not specify any parameter for the algorithm else than__ ``random_state = 1`` __and compute the accuracy of the model built.__

In [8]:
list(df.columns[df.isnull().any()])

['Draught']

In [11]:
# Apply the DecisionTreeClassifier algorithm

df.drop("Draught",axis=1)

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

x = ['shiptype', 'Length', 'Breadth', 'SOG', 'TotalDistance', 'Duration']
y = ['Direct']

X_train, X_test, y_train, y_test = train_test_split(df[x], df[y], test_size = 0.2, random_state = 1)

dt = DecisionTreeClassifier(random_state=1)
dt.fit(X_train, y_train)
predictions = dt.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print(accuracy)

0.97669256381798


The classifier ``DecisionTreeClassifier`` have several parameters that may need tuning. One of them is ``max_depth``, representing the depth of the tree.

_What would be the consequence of a tree with unlimited depth?_

: Might overfit

__Take your code from the previous task, and try out a couple of values for the parameter__ ``max_depth``.

In [ ]:
# Try different values for max_depth
def max_depth_dt(max_depth:int = None):
	dt = DecisionTreeClassifier(random_state=1, max_depth=max_depth)
	dt.fit(X_train, y_train)
	predictions = dt.predict(X_test)
	accuracy = accuracy_score(y_test, predictions)
	print(f"Max Depth: {max_depth}, Accuracy: {accuracy}")

for depth in range(30):
	max_depth_dt(depth + 1)

Max Depth: 1, Accuracy: 0.9722530521642619
Max Depth: 2, Accuracy: 0.9722530521642619
Max Depth: 3, Accuracy: 0.9800221975582686
Max Depth: 4, Accuracy: 0.9822419533851277
Max Depth: 5, Accuracy: 0.9833518312985572
Max Depth: 6, Accuracy: 0.9833518312985572
Max Depth: 7, Accuracy: 0.9811320754716981
Max Depth: 8, Accuracy: 0.9811320754716981
Max Depth: 9, Accuracy: 0.978912319644839
Max Depth: 10, Accuracy: 0.9778024417314095
Max Depth: 11, Accuracy: 0.978912319644839
Max Depth: 12, Accuracy: 0.978912319644839
Max Depth: 13, Accuracy: 0.97669256381798
Max Depth: 14, Accuracy: 0.9778024417314095
Max Depth: 15, Accuracy: 0.97669256381798
Max Depth: 16, Accuracy: 0.97669256381798
Max Depth: 17, Accuracy: 0.97669256381798
Max Depth: 18, Accuracy: 0.97669256381798
Max Depth: 19, Accuracy: 0.97669256381798
Max Depth: 20, Accuracy: 0.97669256381798
Max Depth: 21, Accuracy: 0.97669256381798
Max Depth: 22, Accuracy: 0.97669256381798
Max Depth: 23, Accuracy: 0.97669256381798
Max Depth: 24, Accur

What is the value for ``max_depth`` which gives the best accuracy? __Write it down in the following cell (if several values give the maximum accuracy, just write one of them). (1 point)__

In [ ]:
# Best value for max_depth (integer)

t21_max_depth = 5 #Accuracy: 0.9833518312985572

The parameter ``min_samples_leaf`` gives the minimal number of instances required for a leaf node.

_What would be the consequence of a tree where the leaves can hold 1 instance?_

: Might overfit

__Take your code from the previous task, and try out a couple of values for the parameter__ ``min_samples_leaf``. Do not specify a value for ``max_depth``, keep the default parameter.

In [15]:
# Try different values for min_samples_leaf
def min_samples_leaf_dt(min_samples_leaf:int = None):
	dt = DecisionTreeClassifier(random_state=1, min_samples_leaf=min_samples_leaf)
	dt.fit(X_train, y_train)
	predictions = dt.predict(X_test)
	accuracy = accuracy_score(y_test, predictions)
	print(f"Min Samples Leaf: {min_samples_leaf}, Accuracy: {accuracy}")

for samples in range(30):
	min_samples_leaf_dt(samples + 1)

Min Samples Leaf: 1, Accuracy: 0.97669256381798
Min Samples Leaf: 2, Accuracy: 0.978912319644839
Min Samples Leaf: 3, Accuracy: 0.978912319644839
Min Samples Leaf: 4, Accuracy: 0.978912319644839
Min Samples Leaf: 5, Accuracy: 0.9822419533851277
Min Samples Leaf: 6, Accuracy: 0.9800221975582686
Min Samples Leaf: 7, Accuracy: 0.9822419533851277
Min Samples Leaf: 8, Accuracy: 0.9800221975582686
Min Samples Leaf: 9, Accuracy: 0.9811320754716981
Min Samples Leaf: 10, Accuracy: 0.9822419533851277
Min Samples Leaf: 11, Accuracy: 0.9822419533851277
Min Samples Leaf: 12, Accuracy: 0.9822419533851277
Min Samples Leaf: 13, Accuracy: 0.9822419533851277
Min Samples Leaf: 14, Accuracy: 0.9800221975582686
Min Samples Leaf: 15, Accuracy: 0.9800221975582686
Min Samples Leaf: 16, Accuracy: 0.9800221975582686
Min Samples Leaf: 17, Accuracy: 0.9800221975582686
Min Samples Leaf: 18, Accuracy: 0.9800221975582686
Min Samples Leaf: 19, Accuracy: 0.9800221975582686
Min Samples Leaf: 20, Accuracy: 0.98002219755

What is the value for ``min_samples_leaf`` which gives the best accuracy? __Write it down in the following cell (if several values give the maximum accuracy, just write one of them). (1 point)__

In [16]:
# Best value for min_samples_leaf (integer)

t21_min_sample_leaf = 4 #Accuracy: 0.9822419533851277

Other parameters can be tuned for this algorithm. We won't go further in this exercise, but all the parameters can be found in the documentation of the algorithm.

### 2.2 Cost Complexity Pruning (20 min) <a class="anchor" id="t22"></a>

With the same dataset we used for the rest of the exercise, we will compare cost complexity pruning with the previous approaches.

In [17]:
# Import the dataset

import pandas as pd
df = pd.read_csv('03-decision-tree_1.csv', na_values = '?')
df['Direct'] = df['Direct'].astype('category')
x = ['shiptype', 'Length', 'Breadth', 'SOG', 'TotalDistance', 'Duration']
y = ['Direct']

X_train, X_test, y_train, y_test = train_test_split(df[x], df[y], test_size = 0.2, random_state = 1)

We will use here the algorithm ``Cost Complexity feature`` from the library ``sklearn`` and ``DesisionTreeClassifier``. 

__Write the code for the train and test sets and applying the algorithm (still with__ ``random_state = 1``__). Get the accuracy of your model with the function__ ``accuracy_score``.

Note: used as we did before, the algorithm ``DecissionTreeClassifier`` returns a warning message about the expected format of the parameter ``y`` (in our case, ``y_train``). This is just a warning so it could be ignored, but to suppress it, we can replace ``y_train`` with ``y_train.values.ravel()``. This basically changes the type of the parameters ``y_train`` but has no influence on the model. To verify, you can try to compare the accuracy of both models, with and without the change.


In [18]:
# Import the  libraries
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
import pandas as pd
import numpy as np

First we have to build a complete decision tree as before.

In [19]:
# Build the decision tree
dt = DecisionTreeClassifier(random_state=1)
dt.fit(X_train, y_train)
predictions = dt.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print(accuracy)

0.97669256381798


After having build the decision tree. We can apply cost complexitiy pruning by using the function _cost_complexity_pruning_path_ of the classifier. Input are the feature vector of the training dataset and the vector of the feature that should be predicted.
The function returns a complex structure which we call __path__. You can get the alpha values by __path__['ccp_alphas']

In [20]:
# compute the alpha array of Cost Complexity pruning
path= dt.cost_complexity_pruning_path(X_train, y_train)
alphas= path['ccp_alphas']
alphas

array([0.        , 0.00016195, 0.00018335, 0.00018508, 0.00024986,
       0.0002568 , 0.00025779, 0.00025779, 0.0002644 , 0.0002644 ,
       0.00026555, 0.00027068, 0.00027597, 0.00027752, 0.00027944,
       0.00029613, 0.00036194, 0.00037016, 0.00037016, 0.00037016,
       0.00040221, 0.00040886, 0.00041644, 0.00041644, 0.00041644,
       0.00042084, 0.00046271, 0.00047593, 0.00047611, 0.00051496,
       0.00056056, 0.00069406, 0.00083154, 0.00085524, 0.0008884 ,
       0.00092039, 0.00112335, 0.00165648, 0.00169394, 0.00182045,
       0.0018268 , 0.00209115, 0.00382123, 0.00484766, 0.1025496 ])

In [21]:
# Find the best alpha and tree. 
clfs = []
accs = []

def ccp_alpha_dt(ccp_alpha:float = 0):
	dt = DecisionTreeClassifier(random_state=1, ccp_alpha=ccp_alpha)
	dt.fit(X_train, y_train)
	predictions = dt.predict(X_test)
	accuracy = accuracy_score(y_test, predictions)
	print(f"CCP Alpha: {ccp_alpha}, Accuracy: {accuracy}")
	clfs.append(dt)
	accs.append(accuracy)



for alpha in alphas:
	ccp_alpha_dt(alpha)

CCP Alpha: 0.0, Accuracy: 0.97669256381798
CCP Alpha: 0.00016194706644456782, Accuracy: 0.97669256381798
CCP Alpha: 0.00018335261995326227, Accuracy: 0.97669256381798
CCP Alpha: 0.00018508236165093466, Accuracy: 0.97669256381798
CCP Alpha: 0.0002498611882287619, Accuracy: 0.9778024417314095
CCP Alpha: 0.000256801776790672, Accuracy: 0.9778024417314095
CCP Alpha: 0.00025779328944237337, Accuracy: 0.9811320754716981
CCP Alpha: 0.00025779328944237337, Accuracy: 0.9811320754716981
CCP Alpha: 0.0002644033737870496, Accuracy: 0.9800221975582686
CCP Alpha: 0.0002644033737870496, Accuracy: 0.9800221975582686
CCP Alpha: 0.00026555295367308024, Accuracy: 0.9800221975582686
CCP Alpha: 0.00027068295391449173, Accuracy: 0.9800221975582686
CCP Alpha: 0.0002759710213902323, Accuracy: 0.9800221975582686
CCP Alpha: 0.0002775219233173486, Accuracy: 0.9811320754716981
CCP Alpha: 0.0002794399812656575, Accuracy: 0.9822419533851277
CCP Alpha: 0.0002961317786414954, Accuracy: 0.9833518312985572
CCP Alpha: 0

Write down the alpha and compare the tree (depth, number of nodes, accuracy) with the tree you found with max_depth.

In [22]:
# Comparision

best_idx = np.argmax(accs)
best_tree = clfs[best_idx]

t22_best_alpha = alphas[best_idx]
t22_cost_complexity_accuray = accs[best_idx]
t22_cost_complexity_depth = best_tree.get_depth()
t22_cost_complexity_nodes=best_tree.tree_.node_count


_What is the main difference between choosing max_depth and cost complexity pruning_

: max_depth - cut the tree at a given depth even if it increases the error rate, simple to implement. but cost_complexity_pruning limits or cuts the tree based on actual contribution to accuracy , needs more computation.